### Django authentication system
- 따로 settings에 추가 필요 없음
- 이미 들어가있음('django.contrib.auth',)

- 인증(Authentication)
  - 신원확인
  - 사용자가 자신이 누구인지 확인하는 것
- 권한(Authorization)
  - 권한 부여
  - 인증된 사용자가 수행할 수 잇는 작업을 결정

- 두 번째 앱으로 처리
- accounts -> 회원, 인증

- auth와 관련된 경로나 키워드들을 Django 내부적으로 accounts라는 이름으로 사용하고 있기 때문에 되도록 accounts로 지정하는 것을 권장
- 다른 이름으로 설정해도 되지만 나중에 추가 설정을 필요로 함.(번거로워짐)


### 커스텀 유저 모델로 대체하기
- Django는 기본 user model을 필수적으로 custom user model로 대체
- 기본 User모델로 충분하더라도 커스텀 하도록 "강력하게" 권장
- 대체 커스텀은 migration 하기 전에 마쳐야 함
- 일부 프로젝트에서는 user-model의 기본 인증 요구사항이 적절하지 않을 수 있음
- username 대신 email을 식별로 이용하는 경우 built-in user model에서는 수정하기 쉽지 않다. 따라서 처음부터 커스텀해서 시작하라는 것.

- AbstractUser를 상속받는 커스텀 user 클래스 작성
- 기존 user도 AbstractUser를 상속받기 때문에 이것만 상속받으면 커스텀할 수 있음

